# DeepSpeech benchmarking


In [ ]:
# !pip install deepspeech
!pip install deepspeech-gpu # gpu version
!pip install git+https://github.com/openai/whisper.git
!pip install jiwer
!pip install swifter
!pip install rouge_score
!pip install evaluate

# Score
import jiwer
from whisper.normalizers import EnglishTextNormalizer
from scipy.io import wavfile
from deepspeech import Model
from google.colab import drive
import os
import numpy as np
try:
    import tensorflow as tf  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass
import torch
import pandas as pd
import torchaudio
from tqdm.notebook import tqdm
from deepspeech import Model

# drive.mount('/content/drive')



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 22.3 MB 1.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-sili2k5o
  Running command git clone -q https://github.com/openai/whisper.git /tmp/pip-req-build-sili2k5o
     |████████████████████████████████| 5.5 MB 3.7 MB/s 
     |████████████████████████████████| 7.6 MB 40.6 MB/s 
     |████████████████████████████████| 182 kB 58.1 MB/s 
  Created wheel for whisper: filename=whisper-1.0-py3-none-any.whl size=1175239 sha256=7d06dc7121585c59063752f622e1939c3101af5c623f55c46a4b113551cd3457
  Stored in directory: /tmp/pip-ephem-wheel-cache-yyqooj9o/wheels/16/15/89/1c7bb31bd0006793a95549d04785121a8a36daad9158e1e43a
Successfully built whisper
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=5b1c43e0e62c6b3c893003fee76633f08af57505aab9658531525d0a1484a36d
  Stored in directory: /root/.cache/pip/wheels/84/ac/6b/38096e3c5bf1dc87911e3585875e21a3ac610348e740409c76
Successfully built rouge-score
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 1.3 MB/s 
     |████████████████████████████████| 212 kB 10.8 MB/s 
     |████████████████████████████████| 451 kB 67.9 MB/s 
     |████████████████████████████████| 115 kB 71.0 MB/s 
     |████████████████████████████████| 127 kB 71.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


## Download model

In [ ]:
# # # Download deepspeech pre-trained English model files
!curl -LO https://github.com/mozilla/DeepSpeech/releases/download/v0.9.3/deepspeech-0.9.3-models.pbmm
!curl -LO https://github.com/mozilla/DeepSpeech/releases/download/v0.9.3/deepspeech-0.9.3-models.scorer

# # Download example audio files
!curl -LO https://github.com/mozilla/DeepSpeech/releases/download/v0.9.3/audio-0.9.3.tar.gz
!tar xvf audio-0.9.3.tar.gz

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  180M  100  180M    0     0  20.9M      0  0:00:08  0:00:08 --:--:-- 16.9M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  909M  100  909M    0     0  71.7M      0  0:00:12  0:00:12 --:--:-- 69.4M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  194k  100  194k    0     0   600k      0 --:--:-- --:--:-- --:--:--  600k
._audio
audio/
audio/._2830-3980-0043.wav
audio/2830

'cuda'

## Test model

In [ ]:
# Test the model
!deepspeech --model deepspeech-0.9.3-models.pbmm --scorer deepspeech-0.9.3-models.scorer --audio audio/4507-16021-0012.wav

2022-11-27 21:04:58.893333: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
Loading model from file deepspeech-0.9.3-models.pbmm
TensorFlow: v2.3.0-6-g23ad988
DeepSpeech: v0.9.3-0-gf2e9c85
2022-11-27 21:04:59.033116: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2022-11-27 21:04:59.033286: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcublas.so.10'; dlerror: libcublas.so.10: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2022-11-27 21:04:59.075518: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] C

# Testing with Librispeech

In [ ]:
dataset = torchaudio.datasets.LIBRISPEECH(
    root=os.path.expanduser("~/.cache"),
    # url=".",
    url="test-clean",
    download=True,
)


  0%|          | 0.00/331M [00:00<?, ?B/s]

In [ ]:
n = len(dataset)

texts = []
audios = []
for i in range(n):
    f = dataset.__getitem__(i)

    texts.append(f[2])
    audios.append(f[0])
    # audios[i][0]*10_000

df = pd.DataFrame({"reference": texts, "audio": [np.array(audio) for audio in audios]})
df

,reference,audio
0,HE HOPED THERE WOULD BE STEW FOR DINNER TURNIP...,"[[0.00033569336, 0.00030517578, 0.00036621094,..."
1,STUFF IT INTO YOU HIS BELLY COUNSELLED HIM,"[[0.0010375977, 0.0010986328, 0.00091552734, 0..."
2,AFTER EARLY NIGHTFALL THE YELLOW LAMPS WOULD L...,"[[0.00036621094, 0.00064086914, 0.00076293945,..."
3,HELLO BERTIE ANY GOOD IN YOUR MIND,"[[0.0007019043, 0.0005493164, 0.0008544922, 0...."
4,NUMBER TEN FRESH NELLY IS WAITING ON YOU GOOD ...,"[[0.0005187988, 0.00064086914, 0.0007324219, 0..."
...,...,...
2615,OH TO SHOOT MY SOUL'S FULL MEANING INTO FUTURE...,"[[-0.0011291504, -0.0006713867, -0.00048828125..."
2616,THEN I LONG TRIED BY NATURAL ILLS RECEIVED THE...,"[[-0.0018310547, -0.0017089844, -0.0015563965,..."
2617,I LOVE THEE FREELY AS MEN STRIVE FOR RIGHT I L...,"[[0.00039672852, 0.0004272461, 0.0005187988, 0..."
2618,I LOVE THEE WITH THE PASSION PUT TO USE IN MY ...,"[[-0.0014343262, -0.0011901855, -0.0010681152,..."


In [ ]:
normalizer = EnglishTextNormalizer()
df["reference_norm"] = [normalizer(text) for text in df["reference"]]
df

,reference,audio,reference_norm
0,HE HOPED THERE WOULD BE STEW FOR DINNER TURNIP...,"[[0.00033569336, 0.00030517578, 0.00036621094,...",he hoped there would be stew for dinner turnip...
1,STUFF IT INTO YOU HIS BELLY COUNSELLED HIM,"[[0.0010375977, 0.0010986328, 0.00091552734, 0...",stuff it into you his belly counseled him
2,AFTER EARLY NIGHTFALL THE YELLOW LAMPS WOULD L...,"[[0.00036621094, 0.00064086914, 0.00076293945,...",after early nightfall the yellow lamps would l...
3,HELLO BERTIE ANY GOOD IN YOUR MIND,"[[0.0007019043, 0.0005493164, 0.0008544922, 0....",hello bertie any good in your mind
4,NUMBER TEN FRESH NELLY IS WAITING ON YOU GOOD ...,"[[0.0005187988, 0.00064086914, 0.0007324219, 0...",number 10 fresh nelly is waiting on you good n...
...,...,...,...
2615,OH TO SHOOT MY SOUL'S FULL MEANING INTO FUTURE...,"[[-0.0011291504, -0.0006713867, -0.00048828125...",0 to shoot my soul is full meaning into future...
2616,THEN I LONG TRIED BY NATURAL ILLS RECEIVED THE...,"[[-0.0018310547, -0.0017089844, -0.0015563965,...",then i long tried by natural ills received the...
2617,I LOVE THEE FREELY AS MEN STRIVE FOR RIGHT I L...,"[[0.00039672852, 0.0004272461, 0.0005187988, 0...",i love thee freely as men strive for right i l...
2618,I LOVE THEE WITH THE PASSION PUT TO USE IN MY ...,"[[-0.0014343262, -0.0011901855, -0.0010681152,...",i love thee with the passion put to use in my ...


## Transcribing the audio saved in the dataframe

In [ ]:
def transcribe(audio, model):
    # data = np.array(audio[0]*10_000, dtype="int16")
    data = tf.constant(audio[0]*10_000, dtype="int16")
    # print(data)
    return model.stt(data)

In [ ]:
tf.constant(audios[0][0]*10_000, dtype="int16")

<tf.Tensor: shape=(166960,), dtype=int16, numpy=array([ 3,  3,  3, ..., 21, 21, 15], dtype=int16)>

In [ ]:
from datetime import datetime

#### TRANSCRIBING THE TEST SET ####
#### EXECUTE THIS IN THE HPC, ESTIMATED 4H ####
# import swifter

# ds = Model("deepspeech-0.9.3-models.pbmm",)
# ds.enableExternalScorer("deepspeech-0.9.3-models.scorer")

# # test_length = len(df)
# test_length = 100

# df_test = df[:test_length]

# # With swifter
# # a = datetime.now()
# # transs = df_test.apply(lambda row: transcribe(row["audio"], ds), axis=1)
# # b = datetime.now()
# # t = b-a

# auds = df_test["audio"]

# # Without
# transs = []
# a = datetime.now()
# for audio in tqdm(auds):
#     transs.append(transcribe(audio, ds))
# b = datetime.now()
# t2 = b-a

# # print(f"Swifter: {t}")
# print(f"Normal: {t2}")
# # transs

In [ ]:
# df_trans = pd.DataFrame({"trans_norm": [normalizer(t) for t in transs]})
# df_trans

## Loading transcription computed in HPC

In [ ]:
df_trans = pd.read_csv("https://gitlab.gbar.dtu.dk/s210172/whisper-dl-data/raw/master/other_model_outpus/ds_output.csv", index_col=0)
df_trans

,trans
0,he hoped there would be stew for dinner turnip...
1,stuff it into you his belly counseled him
2,after early nightfall the yellow lamps would l...
3,barty and i good in your mind
4,number ten fresh nelly is waiting on you could...
...,...
2615,oh to shoot my soul's full meaning into future...
2616,then i long tried by natural ills received the...
2617,i love thee freely as men strive for right i l...
2618,i love thee with the passion put to use in my ...


In [ ]:
# predictions = list(df_trans["trans"])
predictions = [normalizer(t) for t in df_trans["trans"]]
df_test = df


## Evaluate

In [ ]:
import evaluate

bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

In [ ]:
wer = jiwer.wer(list(df_test["reference_norm"][:n]), predictions)

bleu_score = bleu.compute(predictions=predictions, references=list(df_test["reference_norm"][:n]) )
rouge_score = rouge.compute(predictions=predictions, references=list(df_test["reference_norm"][:n]) )

print(f"WER: {wer * 100:.2f} %")
print(f"Bleu: {bleu_score}")
print(f"Rouge: {rouge_score}")

WER: 7.18 %
Bleu: {'bleu': 0.8721042206151192, 'precisions': [0.9406047516198704, 0.895717874087955, 0.8555504143355908, 0.8177954788180847], 'brevity_penalty': 0.995293592672033, 'length_ratio': 0.9953046331391073, 'translation_length': 52782, 'reference_length': 53031}
Rouge: {'rouge1': 0.9293216144436869, 'rouge2': 0.8777608034747055, 'rougeL': 0.9286368905893192, 'rougeLsum': 0.9288266316364976}


# Testing on Common Voice

## Loading 1k observation of CommonVoice

In [ ]:
from datasets import load_dataset, DatasetDict, Audio, Dataset

dataset_cv = load_dataset("DTU54DL/dmeo")
dataset_cv

Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/DTU54DL___parquet/DTU54DL--dmeo-76d6a02ee7fb2923/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    test: Dataset({
        features: ['audio', 'sentence', 'accent'],
        num_rows: 1000
    })
    train: Dataset({
        features: ['audio', 'sentence', 'accent'],
        num_rows: 1000
    })
})

In [ ]:
audios = dataset_cv["test"]["audio"]
sentences = dataset_cv["test"]["sentence"]

In [ ]:
audios[0]["array"]


array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        1.25619983e-05, -2.85565995e-05,  3.28748853e-05])

In [ ]:
audios_array = [a["array"] for a in audios]
audios_array

In [ ]:
df = pd.DataFrame(data={"audio": audios_array, "reference": sentences})
df

,audio,reference
0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",It was the time of day when all of Spain slept...
1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Same way you did.
2,"[2.3460731158117695e-12, -7.582711106613332e-1...",Sarah told him that she was there to see her b...
3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Galileo Galilei was the first man who observed...
4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","Hey look, a flying pig!"
...,...,...
995,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",What on earth is the matter?
996,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","Man proposes, God disposes."
997,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","No, we don't laminate our business cards."
998,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",What does he say?


In [ ]:
def transcribe_from_commonvoice(audio, model):
    # data = np.array(audio[0]*10_000, dtype="int16")
    data = tf.constant(audio*10_000, dtype="int16")
    # print(data)
    return model.stt(data)

ds = Model("deepspeech-0.9.3-models.pbmm",)
ds.enableExternalScorer("deepspeech-0.9.3-models.scorer")

n_test = 1000

# trans = df["audio"].apply(lambda x: transcribe_from_commonvoice(x, ds))
trans = []
for aud in tqdm(audios_array):
    transcript = transcribe_from_commonvoice(aud, ds)
    trans.append(transcript)



  0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
normalizer = EnglishTextNormalizer()

df["trans_norm"] = [normalizer(t) for t in trans]
df["reference_norm"] = [normalizer(r) for r in df["reference"]]

In [ ]:
import evaluate

bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

predictions = list(df["trans_norm"])

wer = jiwer.wer(list(df["reference_norm"]), predictions)

bleu_score = bleu.compute(predictions=predictions, references=list(df["reference_norm"]) )
rouge_score = rouge.compute(predictions=predictions, references=list(df["reference_norm"]) )

print(f"WER: {wer * 100:.2f} %")
print(f"Bleu: {bleu_score}")
print(f"Rouge: {rouge_score}")

WER: 43.55 %
Bleu: {'bleu': 0.447574952993506, 'precisions': [0.7039746066795474, 0.5784250960307298, 0.50199203187251, 0.44098662978331027], 'brevity_penalty': 0.8168347078506325, 'length_ratio': 0.8317263544536272, 'translation_length': 7246, 'reference_length': 8712}
Rouge: {'rouge1': 0.6040149796904998, 'rouge2': 0.47641174654722585, 'rougeL': 0.6030405410080569, 'rougeLsum': 0.6021577986756044}
